# 结业设计：使用 SageMaker Studio 和 Amazon SageMaker Python SDK 构建端到端表格式数据机器学习 (ML) 项目

## 环境设置

这里的基本设置代码可以帮助您入门。请先查看并运行这些单元格，以安装程序包并创建变量。

In [ ]:
%%capture
%pip install -U shap
%pip install -U smdebug
%pip install imbalanced-learn
%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install sagemaker
%pip install -U seaborn

In [ ]:
#required-libraries

import boto3
import datetime as datetime
import io
import IPython
import json
import math
import matplotlib.pyplot as plt  # visualization
import numpy as np
import pandas as pd
import pathlib
import re
import sagemaker
import seaborn as sns  # visualization
import statistics
import string
import sys
import time
import zipfile

from imblearn.over_sampling import SMOTE

from sagemaker import clarify
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition
from sagemaker.debugger import CollectionConfig, DebuggerHookConfig, FrameworkProfile, ProfilerConfig, ProfilerRule, Rule, rule_configs
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import Run, load_run
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import TransformInput
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.network import NetworkConfig
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.s3 import S3Uploader
from sagemaker.session import Session
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.transformer import Transformer
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.steps import TuningStep
from sagemaker.xgboost.estimator import XGBoost
from sklearn.model_selection import train_test_split

In [ ]:
#basic-variable-code-and-settings

%matplotlib inline
base_job_name = "capstone-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "sagemaker/capstone"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
s3_client = boto3.client("s3")
sagemaker_session = sagemaker.Session()
save_interval = 5
sns.set(color_codes=True)

## 数据集描述

Amazon Simple Storage Service (Amazon S3) 存储桶中存储了五个表：
- **claims.csv**：一个包含原始索赔数据的表。
- **customers.csv**：一个包含原始客户数据的表。
- **claims_preprocessed.csv**：一个包含处理过的索赔数据的表。
- **customers_preprocessed.csv**：一个包含处理过的客户数据的表。
- **claims_customer.csv**：一个联接了处理过的索赔和客户数据的 **policy_id** 的表。

在本实验中，从 **claims.csv** 和 **customers.csv** 表开始。您将在**挑战 1** 中使用 Amazon SageMaker Data Wrangler 对其进行处理。如果您遇到问题，或者想参考处理过的数据集是什么样的，可以查看预处理过的表。

对于这个数据集，目标是 claims 表中的 **fraud** 列。

claims 表包含以下字段： 

- **policy_id**：保单的唯一 ID。
- **driver_relationship**：关系列表（配偶、自己、孩子、其他、不祥）。
- **incident_type**：报告的事故类型（入室抢劫、车祸、盗窃）。
- **collision_type**：碰撞位置（前面、后面、侧面、不详）。
- **incident_severity**：事故严重性（轻微、严重、报废）。
- **authorities_contacted**：首先联系的机构类型（无、警察、救护、消防）。
- **num_vehicles_involved**：事故所涉车辆数量（1 至 6 辆）。
- **num_injuries**：事故所涉受伤人数（1 至 4 人）。
- **num_witnesses**：事故目击者人数（1 至 5 人）。
- **police_report_available**：是否有警察报告（是或否）。
- **injury_claim**：以 USD 为单位的受伤索赔金额（300 至 576,300 USD）。
- **vehicle_claim**：以 USD 为单位的车辆损坏索赔金额（1000 至 51,051 USD）。
- **total_claim_amount**：受伤和损坏索赔总额（2100 至 588,868 USD）。
- **incident_month**：事故发生的月份（1 至 12 月）。
- **incident_day**：事故发生的当天（1 至 31 日）。
- **incident_dow**：事故发生的星期几（0 至 6，表示从星期日到星期六）。
- **incident_hour**：事故发生的具体时间（0 至 23 时）。
- **fraud**：保单是否是欺诈性的（0 或 1）。

customers 表包含以下字段：

- **policy_id**：保单的唯一 ID。
- **customer_age**：客户的年龄（18 至 70 岁）。
- **months_as_customer**：客户已支付保险费的月数（1 至 495 个月）。
- **num_claims_past_year**：客户在过去一年提出的索赔数量。
- **num_insurers_past_5_years**：客户在过去 5 年投保的保险公司数量。
- **policy_state**：客户居住的州（亚利桑那州、加利福尼亚州、爱达荷州、内华达州、俄勒冈州、华盛顿州）。
- **policy_deductable**：以 USD 为单位的保单免赔额（750 至 1100 USD）。
- **policy_annual_premium**：以 USD 为单位的保单年费（2200 至 3000 USD）。
- **policy_liability**：人身伤害的最高赔偿金额，分为单项和所有人身伤害（15/30、25/50、60/90、100/200）。
- **customer_zip**：客户的邮政编码（83201 至 99362）。
- **customer_gender**：客户的性别（男、女、其他、未知）。
- **customer_education**：客户的学历（高中以下、高中、大专、本科、研究生）。
- **auto_year**：汽车制造年份（2001 至 2020 年）。

您可以通过内部联接使用 **policy_id** 列来联接这两个表。

## 挑战实验导航

本实验设有链接，可供在挑战任务和笔记本末尾的附录之间进行导航。如果您想查看附录中的项目，请选择相关的超链接。如果您想返回至当前正在进行的挑战，请选择附录中相应的任务超链接。

实验结构如下所示：

- 挑战 1：使用 SageMaker Data Wrangler 分析和准备数据集
- 挑战 2：在 SageMaker 特征存储中创建特征组
- 挑战 3：训练模型
- 挑战 4：评估模型是否有偏差
- 挑战 5：批量转换
- 挑战 6：构建自动化管道
- 附录

## 挑战 1：使用 SageMaker Data Wrangler 分析和准备数据集

AnyCompany Consulting 收到了一项请求，要求分析汽车保险欺诈数据集并构建模型来帮助预测新的索赔是否有可能是欺诈性的。该模型有 5,000 条客户记录，其中每项索赔都被标记为欺诈性或非欺诈性。您可以使用这些数据来训练、测试和验证模型，然后再对一批新的记录集合运行推理。

使用 Amazon SageMaker Data Wrangler 分析功能来直观了解重要列中的数据分布情况，并检查各列之间的关联以及目标泄漏情况。接着，构建快速基准模型。然后，使用 SageMaker Data Wrangler 数据处理功能来转换各列，使其更适合训练性能更高的模型。

要完成此任务，您需要完成以下子任务：

- 查看数据。
- 在 Amazon SageMaker Studio 中完成探索性数据分析。
- 使用 Amazon SageMaker Clarify 处理器任务来运行偏差报告。
- 准备数据。

完成本挑战大约需要 *100* 分钟。

### 任务 1.1：查看数据

<a id="task1-1-continue"></a>

访问存储库中存储的表格式汽车保险数据集并查看该数据集的样本。存储库中包含两个未经处理的表。一个关于客户数据，名为 **customers.csv**，另一个关于索赔数据，名为 **claims.csv**。

**提示 1**：未经处理的表位于 **./data/** 文件夹中。

**提示 2**：**claims.csv** 和 **customers.csv** 表是未经处理的表。

花点时间探索这两个表。是否有任何突出显示的字段？ 是否有任何需要仔细预处理的字段？

有关如何查看数据的详细步骤，请参阅*附录*部分中的<a href="#task1-1" target="_self">**查看数据（任务 1.1）**</a>。

在访问了汽车保险欺诈表并查看了数据集的样本之后，即表示您完成了此任务。

In [ ]:
#add_your_task_1_1_code_here



### 任务 1.2：在 SageMaker Studio 中完成探索性数据分析 

<a id="task1-2-continue"></a>

通过查看数据、识别数据集中的潜在问题以及检查是否有任何列与目标之间有很大关联来完成探索性数据分析。您可以在 SageMaker Data Wrangler 和笔记本中探索数据。

具体来说就是花点时间查看以下项目：
- **列直方图**：以可视化格式查看各列，并检查数据集中是哪些类型的值。
- **快速模型**：查看数据集并思考预期的模型结果。
- **特征关联**：查看是否有任何列与目标之间有很大关联。
- **目标泄露**：查看是否有任何数据依赖于目标值。

打开 SageMaker Data Wrangler 将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **capstone.ipynb** 选项卡拖到一边，或者选择（右键单击） **capstone.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在使用 SageMaker Data Wrangler 流时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。探索完 SageMaker Data Wrangler 步骤后，通过选择 **capstone.ipynb** 选项卡返回至笔记本。

如果您在创建新的流文件时收到 **The following instance type is not available: ml.m5.4xlarge.Try selecting a different instance below.**（以下实例类型不可用：ml.m5.4xlarge。请尝试选择下面的其他实例。）错误消息，可以选择其他实例类型。接着，尝试 **ml.m5.8xlarge**。

如果显示 **An error occurred loading this view**（加载此视图时出现错误），请关闭 **untitled.flow** 选项卡，然后从文件浏览器中重新打开流文件。

**提示 1**：探索数据集的方法有很多。首先，打开 SageMaker Data Wrangler 流。您需要将 **claims.csv** 和 **customers.csv** 从名称中包含 **databucket-** 的 S3 存储桶导入到 SageMaker Data Wrangler 中。

**提示 2**：要导入第二个表，请返回至 **Data flow**（数据流），然后选择 **Import**（导入）选项卡，以导入其他数据集。

**提示 3**：**Get data insights**（获取数据见解）和 **Add analysis**（添加分析）是在 SageMaker Data Wrangler 中探索数据的两种方法。在查看了一些数据样本图表后，如果您愿意，可以使用笔记本中的其他绘图工具来分析数据。已安装 **plt** 和 **sns** 库。请随意使用您熟悉的任何分析工具来探索数据集。 

**提示 4**：尝试通过**联接**使用 **policy_id** 来联接这两个表。然后，运行其他见解报告。您可以对这些表使用**内部**联接。

联接后的数据集是否让您获得了更有意义的结果？

有关如何在 SageMaker Studio 中探索数据集的详细步骤，请参阅*附录*部分中的<a href="#task1-2-1" target="_self">**在 SageMaker Studio 中探索数据集（任务 1.2）**</a>。

有关如何在笔记本中探索数据集的详细步骤，请参阅*附录*部分中的<a href="#task1-2-2" target="_self">**在笔记本中探索数据集（任务 1.2）**</a>。

在使用 SageMaker Data Wrangler 处理了数据，并探索了数据集，确定了要执行的处理步骤之后，即表示您完成了此任务。

In [ ]:
#add_your_task_1_2_code_here



### 任务 1.3：使用 SageMaker Clarify 处理器任务来运行偏差报告

<a id="task1-3-continue"></a>

使用 SageMaker Clarify 来运行训练前偏差报告，以捕获数据中的类不平衡。使用 SageMaker Data Wrangler 流在 SageMaker Studio 中运行偏差报告。
1.首先，联接两个表。

- 联接：通过**内部**联接使用 **policy_id** 将 **claims.csv** 联接到 **customers.csv**。

**提示 1**：要创建训练前偏差报告，请向 SageMaker Data Wrangler 流添加一项新的分析，然后对于 **Analysis type**（分析类型），选择 **Bias Report**（偏差报告）。

**提示 2**：您可以多次运行偏差报告，每次选择不同的特征进行分析。

有关如何通过 SageMaker Data Wrangler 联接表的详细步骤，请参阅*附录*部分中的<a href="#task1-3-1" target="_self">**在 SageMaker Studio 中联接表（任务 1.3）**</a>。

有关如何运行训练前偏差报告的详细步骤，请参阅*附录*部分中的<a href="#task1-3-2" target="_self">**运行训练前偏差报告（任务1.3）**</a>。

在运行并查看了训练前偏差报告之后，即表示您完成了此任务。

### 任务 1.4：准备数据

<a id="task1-4-continue"></a>

使用 SageMaker Data Wrangler 准备数据集。重点关注以下转换，但可以随意包括其他转换：

- 编码分类（独热编码）：**authorities_contacted**、**collision_type**、**customer_gender**、**driver_relationship**、**incident_type** 和 **policy_state**。
- 有序编码：**customer_education**、**policy_liability**、**incident_severity** 和 **police_report_available**。
- 将列解析为类型：将 **vehicle_claim** 和 **total_claim_amount** 从 **Float** 解析为 **Long**。
- 管理列（删除列）：**customer_zip**。
- 管理列（移动列）：**fraud**（使用 **Move to start**（移至开头））。
- 管理列（重命名列）：删除 **collision_type_N/A** 和 **driver_relationship_N/A** 中的 **/** 符号，将其改为 **_**。
- 管理列（重命名列）：将 **policy_id_0** 重命名为 **policy_id**。

**提示 1**：使用 SageMaker Data Wrangler 联接功能将 **claims** 表联接到 **customers** 表。

**提示 2**：使用 **policy_id** 列联接两个表。

**提示 3**：使用 **Add transform**（添加转换）选项添加转换。

您认为哪些转换对模型训练的影响最大？

有关如何使用 SageMaker Data Wrangler 准备数据的详细步骤，请参阅*附录*部分中的<a href="#task1-4-1" target="_self">**使用 SageMaker Data Wrangler 准备数据（任务 1.4）**</a>。

如果您想导入一部分处理过的数据，请参阅*附录*部分中的<a href="#task1-4-2" target="_self">**导入一部分处理过的数据（任务 1.4）**</a>。


## 挑战 2：在 SageMaker 特征存储中创建特征组

现在您已经处理了数据集，接下来可以创建特征和特征组，以便在将来的分析中使用。使用 SageMaker 特征存储将这些特征存储在特征组中，并在训练模型时查询这些特征。

要完成此任务，请完成以下所有子任务：

1.将特征导出至 SageMaker 特征存储中。
2.使用 Amazon Athena 查询离线存储中的特征组。

完成本挑战大约需要 *30* 分钟。

### 任务 2.1：将特征导出至 SageMaker 特征存储中

<a id="task2-1-continue"></a>

使用 SageMaker Data Wrangler **导出至**功能创建自定义 Jupyter 笔记本。笔记本将创建特征定义和特征组。笔记本将把记录摄取到特征组中。在笔记本中，完成以下步骤：

- 设置 record 和 event_time 的值。
- 运行笔记本单元格以创建特征组。
- 运行笔记本单元格以确认创建的特征组。
- 运行笔记本单元格以将记录摄取到特征组中。

**提示 1**：所有这些步骤都可以在 SageMaker Studio 中完成。创建完特征组后，您可以返回至此笔记本继续执行任务 2.2。

**提示 2**：添加自定义转换来创建 **event_time** 列。

**提示 3**：添加自定义转换的代码如下所示：
    


In [ ]:
import time
import datetime
import pandas as pd
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


**提示 4**：在 SageMaker Data Wrangler 流的最后，选择 **+** 图标，选择 **Export to**（导出至）选项，然后选择 **SageMaker Feature Store (via JupyterNotebook)**（SageMaker 特征存储 (通过 Jupyter 笔记本)）。

**提示 5**：您可以通过将 **enable_online_store** 值从 **True** 更改为 **False** 来关闭在线存储。

如何使用 SageMaker 特征存储来存储和查询用于训练而非推理的记录？

有关如何使用 **Export to**（导出至）选项创建特征组的详细步骤，请参阅*附录*部分中的<a href="#task2-1" target="_self">**使用 Export to（导出至）选项创建特征组（任务 2.1）**</a>。

在创建了特征组并将数据摄取到了特征组之后，即表示您完成了此任务。

### 任务 2.2：使用 Athena 查询离线存储中的特征组

<a id="task2-2-continue"></a>

使用 Athena 从离线数据存储中提取记录。在下一项挑战中，您需要把这些记录分割成训练、测试和验证数据集。

使用下面提供的代码单元格进行 Amazon Athena API 调用。您可以使用 Amazon Athena 控制台进行查询，但此操作超出了本实验的范围。

**提示 1**：您可以使用 **feature_group.athena_query()** 创建 Athena 查询，并使用 **query.table_name** 获取表名称。

**提示 2**：您可以使用 **query.run(query_string=query_string, output_location=output_location)** 运行查询，并使用 **query.as_dataframe()** 将返回的值读取为数据帧。

如何使用 **event_time** 来跟踪发生在数据集时间线上不同点的特征？

有关如何使用 Athena 从离线数据存储中提取记录的详细步骤，请参阅*附录*部分中的<a href="#task2-2" target="_self">**使用 Athena 从离线存储中提取记录（任务 2.2）**</a>。

在将返回的 Athena 查询保存为了数据帧变量之后，即表示您完成了此任务。

In [ ]:
#add_your_task_2_2_code_here



## 挑战 3：训练模型

您的模型已做好训练准备。将数据分割成训练、测试和验证数据集，并训练模型。

之前针对这些数据运行了 SageMaker Autopilot，得到 **F1** 为 **0.616**，**accuracy**（准确率）为 **0.978**，**AUC** 为 **0.918** 以及 **Recall**（查全率）为 **0.539**。要了解有关 SageMaker Autopilot 生成的指标的更多信息，请参阅“其他资源”部分中的“autopilot-metrics-validation”文档，获取更多信息。

在进行训练和优化时，请努力达到或超过 SageMaker Autopilot 分数，并确认 Amazon SageMaker Debugger 没有报错。

要完成此任务，您需要完成以下子任务：

- 创建实验和运行。
- 将数据分割成训练、测试和验证数据集。
- 配置并运行训练任务。
    - 运行基本训练任务。
    - 在启用了 SageMaker Debugger 的情况下运行训练任务并分析报告（可选）。
- 执行超参数优化。

完成本挑战大约需要 *110* 分钟。


### 任务 3.1：为实验和运行命名

<a id="task3-1-continue"></a>

设置变量来为实验和运行命名。
实验需要 **experiment_name**、**run_name** 和 **description**。

有关如何创建变量的详细步骤，请参阅*附录*部分中的<a href="#task3-1" target="_self">**为实验和运行命名（任务 3.1）**</a>。


In [ ]:
#add_your_task_3_1_code_here



### 任务 3.2：将数据分割成训练、测试和验证数据集

<a id="task3-2-continue"></a>

使用您从 SageMaker 特征存储中查询的特征并将数据分割成训练、测试和验证数据集。

**提示 1**：使用 **np.split** 将数据集分割到三个分区中。

**提示 2**：使用 **to_csv** 创建 CSV 文件并使用 **S3Uploader.upload** 将文件添加到 Amazon S3。

**提示 3**：分割的最终结果应该是一个 **data_inputs** 变量，其值为 **train** 和 **validation**。

有关如何将数据分割成训练、测试和验证数据集的详细步骤，请参阅*附录*部分中的<a href="#task3-2" target="_self">**将数据分割成训练、测试和验证数据集（任务 3.2）**</a>。

在将数据分割成了训练、测试和验证数据集之后，即表示您完成了此任务。

In [ ]:
#add_your_task_3_2_code_here



### 任务 3.3：配置并运行训练任务

<a id="task3-3-continue"></a>

开始您的首项训练任务，设置容器、估算器和超参数。然后，使用 **fit()** 训练模型。如果您想查看更详细的报告，请使用 **DebuggerHookConfig** 启用 SageMaker Debugger。

**提示 1**：使用版本为 **1.5-1** 的 **XGBoost** 容器。

**提示 2**：首先为 **eta**、**gamma**、**max_depth**、**min_child_weight**、**num_round**、**objective** 和 **subsample** 设置超参数。

**提示 3**：使用您之前在挑战 1 中创建的 **data_inputs** 作为训练任务的 **inputs** 值。

**提示 4**：您可以通过设置 inputs 和 experiment_config 来配置训练任务。experiment_config 应该包含 **sagemaker_session**、**run_name** 和 **experiment_name**。

**提示 5**：如果您想使用 SageMaker Debugger，请配置 **DebuggerHookConfig**、**ProfilerConfig** 和 Debugger **rule** 对象。

哪些超参数最有可能对模型的性能和准确率产生很大影响？ 您计划首先优化哪些超参数？

有关如何配置并运行基本训练任务的详细步骤，请参阅*附录*部分中的<a href="#task3-3-1" target="_self">**配置并运行基本训练任务（任务 3.3）**</a>。

有关如何在启用了 Debugger 的情况下配置并运行训练任务以及分析报告的详细步骤，请参阅*附录*部分中的<a href="#task3-3-2" target="_self">**在启用了 SageMaker Debugger 的情况下配置并运行训练任务以及分析报告（任务 3.3）**</a>。

在完成了一项或多项训练任务之后，即表示您完成了此任务。

In [ ]:
#add_your_task_3_3_code_here



### 任务 3.4：超参数优化

<a id="task3-4-continue"></a>

现在您已经完成了一项训练任务并对其进行了分析，接下来可以根据您的发现优化超参数范围，并运行更多训练任务来改进模型。

**提示 1**：首先为 **alpha**、**eta**、**max_depth**、**min_child_weight** 和 **num_round** 设置超参数范围。

**提示 2**：在运行 **HyperparameterTuner** 时，请务必根据您的发现设置 **objective_metric_name** 和 **objective_type**。

**提示 3**：要查看您是否改善了 SageMaker Autopilot 结果，请打开 **SageMaker resources**（SageMaker 资源）中的 **Experiments and runs**（实验和运行）菜单。在运行中，查看 **Metrics**（指标）。**ObjectiveMetric** 应高于 **F1** 分数 (**0.616**)，**validation:auc** 的 **Final value**（最终值）应高于 SageMaker Autopilot 分数 (**0.918**)。

在您优化超参数时，哪个对模型性能的提高最大？

有关如何配置训练超参数范围的详细步骤，请参阅*附录*部分中的<a href="#task3-4" target="_self">**配置训练超参数范围（任务 3.4）**</a>。

在配置了训练超参数范围并开始了更多训练任务之后，即表示您完成了此任务。

In [ ]:
#add_your_task_3_4_code_here



## 挑战 4：评估模型是否有偏差

现在您的模型已经训练好了，接下来可以使用 Amazon SageMaker Clarify 评估您的模型。如果您发现任何问题，可以消除检测到的不平衡并重新训练模型。

要完成此任务，您需要完成以下子任务：

- 从训练任务创建模型。
- 创建 SageMaker Clarify 模型配置。
- 创建 SageMaker Clarify 偏差配置。
- 使用 SageMaker Clarify 处理器任务运行偏差、数据和模型报告。
- 消除使用 SageMaker Clarify 检测到的不平衡（可选）。
- 重新训练模型（可选）。

完成本挑战大约需要 *80* 分钟。

### 任务 4.1：从训练任务创建模型

<a id="task4-1-continue"></a>

创建 XGBoost 模型，使用您定义的 **model_name**、**role** 和 **container_def** 调用 **create_model**。

**提示 1**：调用 **xgb.create_model()** 并为模型选择名称。

**提示 2**：使用会话并调用 **create_model**，传入 **model_name**、**role** 和 **container_def**。

有关如何创建模型的详细步骤，请参阅*附录*部分中的<a href="#task4-1" target="_self">**创建模型（任务 4.1）**</a>。

在创建了模型之后，即表示您完成了此任务。

In [ ]:
#add_your_task_4_1_code_here



### 任务 4.2：创建 SageMaker Clarify 模型配置

<a id="task4-2-continue"></a>

使用 **SageMakerClarifyProcessor** 创建 SageMaker Clarify 模型配置。

**提示 1**：设置 **instance_count** 和 **instance_type**。

**提示 2**：使用在结业设计实验开始时创建的 **role** 和 **session**。

有关如何创建 SageMaker Clarity 模型配置的详细步骤，请参阅*附录*部分中的<a href="#task4-2" target="_self">**创建 SageMaker Clarity 模型配置（任务 4.2）**</a>。

在创建了 SageMaker Clarify 模型配置之后，即表示您完成了此任务。

In [ ]:
#add_your_task_4_2_code_here



### 任务 4.3：创建 SageMaker Clarify 偏差配置

<a id="task4-3-continue"></a>

创建数据配置、模型配置、标签配置和偏差配置。

**提示 1**：首先，创建 **DataConfig**，设置输入路径、输出路径、标题和数据集类型。

**提示 2**：然后，创建 **ModelConfig**，选择内容和接受类型、模型名称、实例类型以及实例数目。

**提示 3**：接着，创建 **ModelPredictedLabelConfig**，设置概率阈值。

**提示 4**：最后，创建 **BiasConfig**，设置标签值或阈值、分面名称以及分面值或阈值。

在偏差报告中，您想首先探索哪些分面？ 是否有任何特征特别容易受到偏差的影响？

有关如何创建 SageMaker Clarity 偏差配置的详细步骤，请参阅*附录*部分中的<a href="#task4-3" target="_self">**创建 SageMaker Clarity 偏差配置（任务 4.3）**</a>。

在创建了 SageMaker Clarify 偏差配置之后，即表示您完成了此任务。

In [ ]:
#add_your_task_4_3_code_here



### 任务 4.4：使用 SageMaker Clarify 处理器任务运行偏差、数据和模型报告

<a id="task4-4-continue"></a>

您已为偏差、数据和模型报告选择了所有配置。现在，运行报告。

**提示 1**：将 **data_config**、**bias_config**、**model_predicted_label_config** 和 **model_config** 值传递给 **run_bias**。

**提示 2**：您需要设置 **pre_training_methods** 和 **post_training_methods**。

有关如何使用 SageMaker Clarify 运行偏差、数据和模型报告的详细步骤，请参阅*附录*部分中的<a href="#task4-4" target="_self">**使用 SageMaker Clarify 运行偏差、数据和模型报告（任务 4.4）**</a>。

在使用了 SageMaker Clarify 处理器任务运行偏差、数据和模型报告之后，即表示您完成了此任务。

In [ ]:
#add_your_task_4_4_code_here



### 任务 4.5：消除使用 SageMaker Clarify 检测到的不平衡（可选）

<a id="task4-5-continue"></a>

消除使用 SageMaker Clarify 检测到的不平衡有很多方法。请使用您熟悉的任何方法。本实验中提供了一个合成少数类过采样技术 (SMOTE) 示例，用于消除某列的偏差。

**提示 1**：消除完不平衡后，如果您希望重新测试，请创建一个重新采样的新数据帧。在下一个任务中，您将创建并上传一个新的 CSV 文件。

有关如何消除不平衡的详细步骤，请参阅*附录*部分中的<a href="#task4-5" target="_self">**消除不平衡（任务 4.5）**</a>。

在消除了使用 SageMaker Clarify 检测到的所有不平衡之后，即表示您完成了此任务。

In [ ]:
#add_your_task_4_5_code_here



### 任务 4.6：重新训练模型（可选）

<a id="task4-6-continue"></a>

将新文件上传到 Amazon S3。然后，创建一个新的估算器并使用新数据重新训练。

**提示 1**：使用 **s3_client.upload_file** 将新文件上传到您的存储桶。

**提示 2**：使用 **xgboost_starter_script.py** 并调用 **XGBoost**。然后，使用新数据重新训练。

重新训练后的模型是否获得了更高的 F1 分数？ 使用 SageMaker Debugger 能否解决发现的所有问题？

有关如何重新训练模型的详细步骤，请参阅*附录*部分中的<a href="#task4-6" target="_self">**重新训练模型（任务 4.6）**</a>。

在重新训练了模型之后，即表示您完成了此任务。

In [ ]:
#add_your_task_4_6_code_here



## 挑战 5：批量转换

您的模型已经做好了部署准备。使用具有批量记录的批量转换任务，并在 Amazon S3 中查看预测和准确率数据。然后，清理一些 SageMaker 实例。

要完成此任务，您需要完成以下子任务：

- 为模型创建批量转换任务。
- 在 Amazon S3 中查看预测数据。
- 清理 SageMaker 实例（可选）。

完成本挑战大约需要 *40* 分钟。

### 任务 5.1：为模型创建批量转换任务

<a id="task5-1-continue"></a>

在模型估算器上使用**转换器**创建批量转换任务。然后，运行该批量任务。

**提示 1**：使用转换器并配置 **instance_count**、**instance_type**、**strategy**、**assemble_with** 和 **output_path**。

**提示 2**：将 **test_path** 中列出的测试数据发送到终端节点并等待结果。

有关如何创建批量转换任务的详细步骤，请参阅*附录*部分中的<a href="#task5-1" target="_self">**创建批量转换任务（任务 5.1）**</a>。

在创建了批量转换任务并使用一组记录运行了该任务之后，即表示您完成了此任务。

In [ ]:
#add_your_task_5_1_code_here



### 任务 5.2：在 Amazon S3 中查看预测数据

<a id="task5-2-continue"></a>

完成批量转换任务后，请从 Amazon S3 读取数据。

**提示 1**：您可以使用 **%aws s3 cp --recursive $transformer.output_path ./** 从转换器输出中复制数据

**提示 2**：获得数据后，您可以使用 **%head test_data_batch.csv.out** 进行查看

查看预测。是否有任何出乎意料的预测？

有关如何查看来自批量转换任务的预测数据的详细步骤，请参阅*附录*部分中的<a href="#task5-2" target="_self">**查看来自批量转换任务的预测数据（任务 5.2）**</a>。

在查看了来自批量转换任务的预测数据之后，即表示您完成了此任务。

In [ ]:
#add_your_task_5_2_code_here



### 任务 5.3：清理 SageMaker 实例（可选）

为了降低成本，最佳实践是删除不再使用的实例。您可以使用 SageMaker Studio 快速删除实例。现在花点时间在 SageMaker Studio 中打开当前资源列表，并关闭所有剩余实例。

如果您计划完成下一个管道任务，**请让笔记本实例保持运行**。

**提示 1**：您可以通过在 SageMaker Studio 中选择**正在运行的终端和内核**图标来查看正在运行的实例的列表。

**提示 2**：您可以使用**关闭**图标停止实例。

<a id="task5-3-continue"></a>

有关如何在 SageMaker Studio 中清理 SageMaker 实例的详细步骤，请参阅*附录*部分中的<a href="#task5-3" target="_self">**在 SageMaker Studio 中清理 SageMaker 实例（任务 5.3）**</a>。

当在 SageMaker Studio 中停止了所有 SageMaker 实例之后，即表示您完成了此任务。

## 挑战 6：构建自动化管道（可选）

现在您已经将 Amazon SageMaker Python SDK 和 Amazon SageMaker Studio 用于机器学习 (ML) 工作流，接下来可以使用 SageMaker Pipelines 来扩展您的工作流。在实验环境中演练提供的管道脚本即可完成本挑战。

- 创建管道步骤。
    - 查询 SageMaker 特征存储中处理过的数据。
    - 训练和优化模型。
    - 评估训练后的模型。
    - 执行批量转换任务。
    - 注册模型。
    - 使用 SageMaker Clarify 评估模型训练。
- 定义并启动管道。
- 查看 ML 沿袭跟踪。

完成本挑战大约需要 *120* 分钟。


### 任务 6.1：配置管道

<a id="task6-1-continue"></a>

使用管道模板并配置输入和输出。配置就绪后，运行管道。您的管道可以包括广泛的步骤。下面是建议配置的步骤列表：
- **AutoModelProcess**：在 .csv 文件中拉取数据并将其分割成训练、测试和验证数据集的**处理**步骤。
- **AutoHyperParameterTuning**：使用一系列超参数并优化模型的**优化**步骤。
- **AutoEvalBestModel**：创建评估报告来描述最佳模型的**处理**步骤。
- **CheckAUCScoreAutoEvaluation**：根据评估指标评估模型的**条件**步骤。
- **AutoCreateModel**：创建模型的**模型**步骤。
- **RegisterAutoModel-RegisterModel**：注册模型的**注册模型**步骤。
- **AutoModelConfigFile**：创建偏差报告的**处理**步骤。
- **AutoTransform**：运行批量转换任务的**转换**步骤。
- **ClarifyProcessingStep**：运行 SageMaker Clarify 任务的**处理**步骤。

**提示 1**：有许多管道步骤可供选择。要了解有关管道步骤的更多信息并查看每个步骤的示例代码，请参阅“其他资源”部分中的“构建和管理管道步骤”文档，获取更多信息。

**提示 2**：首先，创建**处理**步骤以拉取数据。然后，创建**优化**步骤以优化模型。接着，创建**模型**步骤以创建模型。

**提示 3**：详细步骤中包含一种示例解决方案，其中包括创建评估报告和偏差报告、运行批量转换任务和 SageMaker Clarify 任务的步骤。

有关如何配置管道的详细步骤，请参阅*附录*部分中的<a href="#task6-1" target="_self">**配置管道（任务 6.1）**</a>。

在设置了管道并开始了管道任务之后，即表示您完成了此任务。

In [ ]:
#add_your_task_6_1_code_here



### 任务 6.2：监控管道

<a id="task6-2-continue"></a>

监控正在运行的管道，查看输入和输出。

**提示 1**：使用 `RunPipeline.describe()` 描述您刚刚创建的管道。

**提示 2**：您可以查看在 SageMaker Studio UI 中运行的管道步骤。打开 **SageMaker resources**（SageMaker 资源）菜单，选择 **Pipelines**（管道），然后选择您创建的管道。

有关如何监控管道的详细步骤，请参阅*附录*部分中的<a href="#task6-2" target="_self">**监控管道**</a>。

在监控完管道之后，即表示您完成了此任务。

In [ ]:
#add_your_task_6_2_code_here



恭喜！ 您使用了一个汽车保险数据集来检测可能存在欺诈的索赔。您探索了一种技术解决方案，使用 SageMaker Studio 和 Amazon SageMaker Python SDK 预测给定汽车保险索赔存在欺诈的可能性。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。

## 其他资源

- [Autopilot 指标](https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html)
- [处理步骤](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)

## 附录

<a name="task1-1" id="task1-1"></a>

### 附录：查看数据（任务 1.1）

要查看数据，请使用 Pandas 指定路径并加载数据。花点时间查看这两个表的样本。

In [ ]:
#read-csv-files
claims_data = pd.read_csv("./data/claims_preprocessed.csv", index_col=0)
customers_data = pd.read_csv("./data/customers_preprocessed.csv", index_col=0)

In [ ]:
#claims-data-sample
claims_data.head()

In [ ]:
#customers-data-sample
customers_data.head()

要继续本实验，请返回至<a href="#task1-1-continue" target="_self">任务 1.1</a>。

<a name="task1-2-1" id="task1-2-1"></a>

### 附录：在 SageMaker Studio 中探索数据集（任务 1.2）

在 SageMaker Data Wrangler 中开始进行数据探索。从 S3 存储桶导入文件并分析数据。

下一个步骤将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **capstone.ipynb** 选项卡拖到一边，或者选择 **capstone.ipynb** 选项卡，然后从工具栏中选择 **File**（文件）和 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览特征组时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。

1.在 SageMaker Studio 的左侧，选择**主页**图标。
1.展开 **Data**（数据）部分，然后选择 **Data Wrangler**。

此时 SageMaker Studio 将打开 **Data Wrangler** 选项卡。

1.选择 **+** **Create Data Wrangler flow**（+ 创建 Data Wrangler 流）。

此时 SageMaker Studio 将打开 **untitled.flow** 选项卡。

1.等待 **untitled.flow** 选项卡完成加载，您可以通过进度条查看加载情况。这可能需要 2–3 分钟。

加载完成后，SageMaker Studio 将在 *Data Wrangler* 选项卡中打开 **Create connection**（创建连接）页面。

1.打开 **untitled.flow** 文件选项卡上的上下文（右键单击）菜单，然后选择 **Rename Data Wrangler Flow...**（重命名 Data Wrangler 流...）以更改文件名。

此时 SageMaker Studio 将打开 **Rename File**（重命名文件）消息窗口。

1.对于 **New Name**（新名称），输入 `CapstoneDataWrangler.flow`。
1.选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Rename</span>（重命名）。

此时 **Rename File**（重命名文件）消息窗口将关闭。

1.在 *CapstoneDataWrangler.flow* 选项卡中的 **Data sources**（数据源）部分，选择 **Amazon S3**。

此时 SageMaker Studio 将在 *DataWrangler.flow* 选项卡中打开 **Import a dataset from S3**（从 S3 导入数据集）页面。

1.在存储桶列表中，打开名称中包含 **databucket** 的存储桶。
1.选择第一个数据集，即名为 **claims.csv** 的文件。

1.选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span>（导入）。

1.返回至 **Data flow**（数据流）视图，选择 *CapstoneDataWrangler.flow* 选项卡左上角的 **&lt; 数据流**图标。
1.选择 *CapstoneDataWrangler.flow* 选项卡左上角的 **Import**（导入）选项卡。

此时 SageMaker Studio 将打开 **Create connection**（创建连接）页面。

1.在 *CapstoneDataWrangler.flow* 选项卡中的 **Data sources**（数据源）部分，选择 **Amazon S3**。

此时 SageMaker Studio 将在 *DataWrangler.flow* 选项卡中打开 **Import a dataset from S3**（从 S3 导入数据集）页面。

1.在存储桶列表中，打开名称中包含 **databucket** 的存储桶。
1.选择第二个数据集，即名为 **customers.csv** 的文件。
1.选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span>（导入）。

1.返回至 **Data flow**（数据流）视图，选择 *CapstoneDataWrangler.flow* 选项卡左上角的 **&lt; 数据流**图标。

1.在 <b>Data flow</b>（数据流）选项卡中，选择**数据类型**图标旁边的 **+** 符号，然后选择 **Get data insights**（获取数据见解）。

1.创建报告并探索见解。

1.返回至 **Data flow**（数据流）选项卡，选择**数据类型**图标旁边的 **+** 符号，然后选择 **Add analysis**（添加分析）。

1.创建分析并探索结果。

使用任何可以帮助您全面探索数据集的报告类型。完成后，您可以继续执行下一个任务。

要继续本实验，请返回至<a href="#task1-2-continue" target="_self">任务 1.2</a>。


<a name="task1-2-2" id="task1-2-2"></a>

### 附录：在笔记本中探索数据集（任务 1.2）

探索数据集的方法有很多。下面是您可以采取的一些数据探索步骤的示例。使用这些示例作为参考来开始探索数据集的各个方面。

In [ ]:
#gender-graph
import matplotlib.pyplot as plt
customers_data.customer_gender_female.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Male", "Female"]);

In [ ]:
#fraud-graph
claims_data.fraud.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Not Fraud", "Fraud"]);

In [ ]:
#education-category-graphs
educ = customers_data.customer_education.value_counts(normalize=True, sort=False)
plt.bar(educ.index, educ.values)
plt.xlabel("Customer Education Level");

In [ ]:
#claim-amount-graph
plt.hist(claims_data.total_claim_amount, bins=30)
plt.xlabel("Total Claim Amount")

In [ ]:
#claims-filed-graph
customers_data.num_claims_past_year.hist(density=True)
plt.suptitle("Number of Claims in the Past Year")
plt.xlabel("Number of claims per year")

In [ ]:
#paid-plot-graphs
sns.pairplot(
    data=customers_data, vars=["num_insurers_past_5_years", "months_as_customer", "customer_age"]
);

In [ ]:
#fraud-insurers-graph
combined_data = customers_data.join(claims_data)
sns.lineplot(x="num_insurers_past_5_years", y="fraud", data=combined_data);

In [ ]:
#months-as-customer-graph
sns.boxplot(x=customers_data["months_as_customer"]);

In [ ]:
#customer-age-graph
sns.boxplot(x=customers_data["customer_age"]);

In [ ]:
#fraud-gender-graph
combined_data.groupby("customer_gender_female").mean()["fraud"].plot.bar()
plt.xticks([0, 1], ["Male", "Female"])
plt.suptitle("Fraud by Gender");

In [ ]:
#correlation-matrix-graph
cols = [
    "fraud",
    "customer_gender_male",
    "customer_gender_female",
    "months_as_customer",
    "num_insurers_past_5_years",
]
corr = combined_data[cols].corr()

# plot the correlation matrix
sns.heatmap(corr, annot=True, cmap="Reds");

In [ ]:
#load-combined-data
combined_data = pd.read_csv("./data/claims_customer.csv")

In [ ]:
#remove-unnecessary-columns
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains("^Unnamed: 0")]
combined_data.head()

In [ ]:
#describe-combined-data
combined_data.describe()

In [ ]:
#generate-statistics
combined_stats = []

for col in combined_data.columns:
    combined_stats.append(
        (
            col,
            combined_data[col].nunique(),
            combined_data[col].isnull().sum() * 100 / combined_data.shape[0],
            combined_data[col].value_counts(normalize=True, dropna=False).values[0] * 100,
            combined_data[col].dtype,
        )
    )

stats_df = pd.DataFrame(
    combined_stats,
    columns=["feature", "unique_values", "percent_missing", "percent_largest_category", "datatype"],
)
stats_df.sort_values("percent_largest_category", ascending=False)

In [ ]:
#heatmap-graph
sns.set_style("white")

corr_list = [
    "customer_age",
    "months_as_customer",
    "total_claim_amount",
    "injury_claim",
    "vehicle_claim",
    "incident_severity",
    "fraud",
]

corr_df = combined_data[corr_list]
corr = round(corr_df.corr(), 2)

fix, ax = plt.subplots(figsize=(15, 15))

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(corr, mask=mask, ax=ax, annot=True, cmap="OrRd")

ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10, ha="right", rotation=45)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10, va="center", rotation=0)

plt.show()

要继续本实验，请返回至<a href="#task1-2-continue" target="_self">任务 1.2</a>。

<a name="task1-3-1" id="task1-3-1"></a>

### 附录：在 SageMaker Studio 中联接表（任务1.3）

1.返回至 **Data flow**（数据流）视图，选择 *CapstoneDataWrangler.flow* 选项卡左上角的 **&lt; 数据流**图标。
1.选择 **claims.CSV 数据类型**图标旁边的 **+** 符号，然后从上下文菜单中选择 **Join**（联接）。

此时 SageMaker Data Wrangler 将显示 **Join**（联接）页面。

1.选择 **customers.csv 数据类型**图标。
1.选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span>（配置）。
1.对于 **Join Type**（联接类型），选择 **Inner**（内部）。
1.在 **Columns**（列）部分：

    - 对于 **Left**（左侧），选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>。
    
    - 对于 **Right**（右侧），选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>。

1.选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>（预览）。

1.选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>（添加）。

要继续本实验，请返回至<a href="#task1-3-continue" target="_self">任务 1.3</a>。

<a name="task1-3-2" id="task1-3-2"></a>

### 附录：运行训练前偏差报告（任务 1.3）

使用 SageMaker Data Wrangler 流创建 SageMaker Clarify 偏差报告。

1.选择 **CapstoneDataWrangler.flow** 选项卡。
1.前往 **Data flow**（数据流）视图。如有必要，请选择 *DataWranglerLab.flow* 选项卡左上角的 **&lt; 数据流**图标。
1.选择**联接**图标旁边的 **+** 符号，然后从上下文菜单中选择 **Add analysis**（添加分析）。
1.在 **Create analysis**（创建分析）部分：

- 对于 Analysis type（分析类型），选择 **Bias Report**（偏差报告）。
- 对于 **Analysis name**（分析名称），输入 `fraud bias by age`。
–对于 **Select the column your model predicts (target)**（选择您的模型预测的列 (目标)），选择 **fraud**（欺诈）。
- 对于 **Is your predicted column a value or threshold?**（您的预测列是一个值还是阈值?），选择 **value**（值）选项。
- 对于 **Predicted value(s)**（预测值），输入 **1**。
–对于 **Select the column to analyze for bias**（选择要分析偏差的列），选择 **customer_age**。

1.选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Check for bias</span>（检查偏差）。

在任务完成后，查看返回的指标。注意是否有偏差，并计划您希望对所分析的列采取的任何处理步骤。

1.对于任何您想要保存的分析，请选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Save</span>（保存）。

您可以对想要分析偏差的任何列重复这些步骤。

要继续本实验，请返回至<a href="#task1-3-continue" target="_self">任务 1.3</a>。

<a name="task1-4-1" id="task1-4-1"></a>

### 附录：使用 SageMaker Data Wrangler 准备数据（任务 1.4）

合并数据集，在 SageMaker Data Wrangler 中使用 **policy_id** 进行联接。

使用 SageMaker Data Wrangler，您可以在流中的任何位置联接数据。您可以在联接之前完成各个文件的数据准备，也可以在联接之后转换特征。SageMaker Data Wrangler 流是灵活的。

如果您还没有在任务 1.3 中联接表，下面的步骤将指导您完成这个过程。

1.返回至 **Data flow**（数据流）视图，选择 *CapstoneDataWrangler.flow* 选项卡左上角的 **&lt; 数据流**图标。
1.选择**数据类型**图标旁边的 **+** 符号，然后从上下文菜单中选择 **Join**（联接）。

此时 SageMaker Data Wrangler 将显示 **Join**（联接）页面。

1.选择第二个**数据类型**图标。
1.选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span>（配置）。
1.对于 **Join Type**（联接类型），选择 **Inner**（内部）。
1.在 **Columns**（列）部分：

    - 对于 **Left**（左侧），选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>。
    
    - 对于 **Right**（右侧），选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>。

1.选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>（预览）。

1.选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>（添加）。

联接数据表后，转换合并后的数据。

1.返回至 **Data flow**（数据流）视图，选择 *CapstoneDataWrangler.flow* 选项卡左上角的 **&lt; 数据流**图标。
1.选择**联接**图标旁边的 **+** 符号，然后从上下文菜单中选择 **Add transform**（添加转换）。

可以使用此菜单向数据集添加多项转换。数据集的预览将显示在转换菜单的左侧。

将以下转换步骤添加到 SageMaker Data Wrangler 流中：
- 编码分类（独热编码）：**authorities_contacted**、**collision_type**、**customer_gender**、**driver_relationship**、**incident_type** 和 **policy_state**，使用 **Skip**（跳过）无效处理策略，选择输出样式为 **Columns**（列）。
- 编码分类（有序编码）：**customer_education**、**incident_severity**、**police_report_available** 和 **policy_liability**，使用 **Skip**（跳过）无效处理策略。
- 将列解析为类型：将 **vehicle_claim** 和 **total_claim_amount** 从 **Float** 解析为 **Long**。
- 管理列（删除列）：**customer_zip** 和 **policy_id_1**。
- 管理列（移动列）：**fraud**（使用 **Move to start**（移至开头））。
- 管理列（重命名列）：将 **collision_type_N/A** 和 **driver_relationship_N/A** 中的 **/** 符号替换为 **_**。
- 管理列（重命名列）：将 **policy_id_0** 重命名为 **policy_id**。

如果任何列名中有 **/** 字符，则重命名该列，将 **/** 替换为 **_**。如果任何列名中有空白格字符，则重命名该列，将空白格替换为 **_**。例如，对于使用独热编码创建的任何列，如果值为 **N/A**，则需要重命名该列。SageMaker 特征存储不接受包含 **/** 或空白格字符的列。

如果您已转换数据并做好了开始训练模型的准备，可以继续执行下一个任务。您可以随时返回至此流，并根据您在训练和优化期间的发现进行更改。


要继续本实验，请返回至<a href="#task1-4-continue" target="_self">任务 1.4</a>。

<a name="task1-4-2" id="task1-4-2"></a>

### 附录：导入一部分处理过的数据（任务 1.4）

如果您在预处理过程中遇到问题，或者希望加载一组已经为您处理过的数据，请访问存储在 S3 存储桶的数据文件夹中的处理过的数据。

In [ ]:
#processed-data-import
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key=f"{prefix}/data/raw/claims_customer.csv")
df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
df_processed.head()

要继续本实验，请返回至<a href="#task1-4-continue" target="_self">任务 1.4</a>。

<a name="task2-1" id="task2-1"></a>

### 附录：使用 <b>Export to</b>（导出至）选项创建特征组（任务 2.1）

SageMaker Data Wrangler 可以将数据导出至 SageMaker 特征存储。它会创建一个笔记本，其中包含配置特征组所需的所有代码，并会将转换后的数据摄取到特征组中。

1.选择 **Add step**（添加步骤）。

1.选择 **Custom transform**（自定义转换）。

1.对于 **Name**（名称），输入 `event_time`。

1.选择 **Python (PySpark)**（如果未选择）。

1.对于 **Your custom transform**（您的自定义转换），输入以下内容：


In [ ]:
import time
import datetime
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


1.选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>（预览）。

1.选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>（添加）。

这会将 **event_time** 作为列添加到您的数据集中。SageMaker 特征存储需要 **event_time** 和唯一 **record** ID。使用 **policy_id** 作为 **record** ID。

1.要返回至数据流，请选择 **&lt; 数据流**图标。

1.在 SageMaker Data Wrangler 中选择转换旁边的 **+** 图标。

1.选择 **Export to**（导出至）。

1.选择 **SageMaker Feature Store (via Jupyter Notebook)**（SageMaker 特征存储 (通过 Jupyter 笔记本)）。

此时将打开一个新的笔记本。

1.在第一个单元格中，更改以下变量：
- 对于 **record_identifier_feature_name**，将 **None** 替换为 `"policy_id"`。如果您联接了 customers 表和 claims 表，并且没有删除第二个 **policy_id** 列，则可能需要将 **None** 替换为 `"policy_id_0"`。请勿更改 **if** 语句后面的 **None** 值。
- 对于 **event_time_feature_name**，将 **None** 替换为 `"event_time"`。请勿更改 **if** 语句后面的 **None** 值。

**预计输出**：当您完成单元格的编辑之后，它应该与以下示例类似：


In [ ]:
record_identifier_feature_name = "policy_id"
if record_identifier_feature_name is None:
   raise SystemExit("Select a column name as the feature group record identifier.")

event_time_feature_name = "event_time"
if event_time_feature_name is None:
   raise SystemExit("Select a column name as the event time feature name.")


2.运行所有单元格以创建特征定义和特征组，并使用处理任务将转换后的数据摄取到特征组中。

当单元格完成后，您即可使用特征存储。

要继续本实验，请返回至<a href="#task2-1-continue" target="_self">任务 2.1</a>。

<a name="task2-2" id="task2-2"></a>

### 附录：使用 Athena 从离线存储中提取记录（任务 2.2）

使用 **athena_query** 设置 Athena 查询。然后，设置 **query_string**。最后，运行查询并查看结果示例。

In [ ]:
#configure-and-run-athena-query
try:
    # If there is a feature group, get the name
    feature_group_name = sagemaker_session.boto_session.client("sagemaker", region_name=region).list_feature_groups()['FeatureGroupSummaries'][0]['FeatureGroupName']
    feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

    # Confirm the Athena settings are configured
    try:
        boto3.client('athena').update_work_group(
            WorkGroup='primary',
            ConfigurationUpdates={
                'EnforceWorkGroupConfiguration':False
            }
        )
    except Exception:
        pass
    
    # Configure the query
    query = feature_group.athena_query()
    table = query.table_name
    query_string = f'SELECT * FROM "{table}" '
    output_location = f"s3://{sagemaker_session.default_bucket()}/query_results/"
    print(f"Athena query output location: \n{output_location}")

    # Run the query
    query.run(query_string=query_string, output_location=output_location)
    query.wait()
    df_feature_store = query.as_dataframe()
    
    # Wait for data to appear in the feature group
    attempts = 0
    while len(df_feature_store.index) == 0 and attempts < 30:
        print("Waiting for feature group to populate...")
        time.sleep(60)
        # Rerun the query
        query.run(query_string=query_string, output_location=output_location)
        query.wait()
        df_feature_store = query.as_dataframe()
        # Increment the attempts
        attempts += 1
    if len(df_feature_store.index) != 0:
        print("The feature group is populated.")
except IndexError as e:
    # If there is no feature group, thrown an error
    print("No feature groups were found. Please create a feature group.")

运行之前的代码块后，Amazon Athena 查询记录就保存在名称以 *sagemaker* 开头的 Amazon S3 存储桶中。保存的查询对象位于名为 **query_results** 的目录中。

要继续本实验，请返回至<a href="#task2-2-continue" target="_self">任务 2.2</a>。

<a name="task3-1" id="task3-1"></a>

### 附录：为实验和运行命名（任务 3.1）

要创建实验，请使用库 **sagemaker.experiments.run**。设置 **experiment_name**、**run_name** 和 **description**。

In [ ]:
from time import gmtime, strftime

#create experiment and run-names
create_date = strftime("%m%d%H%M")
capstone_experiment_name="capstone-experiment-{}".format(create_date)
capstone_run_name = "lab-capstone-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-capstone', 'Value': 'lab-capstone-run'}]

# provide a description
description="Using SM Experiments with the Auto dataset."

print(f"Experiment name - {capstone_experiment_name},  run name - {capstone_run_name}")

要继续本实验，请返回至<a href="#task3-1-continue" target="_self">任务 3.1</a>。

<a name="task3-2" id="task3-2"></a>

### 附录：将数据分割成训练、测试和验证数据集（任务 3.2）

要分割数据，请使用 **np.split** 并指定要如何分割数据。然后，创建 CSV 文件并将其上传到 S3 存储桶。接着，设置训练输入。最后，创建 **data_inputs** 变量。在整个挑战中使用 data_inputs，以在训练模型时指定训练和验证数据集。

In [ ]:
#train-validation-test-split
try:
    # If there is a feature group, use it
    df_feature_store = df_feature_store.iloc[: , :-4]
    df_processed_pre_split = df_feature_store
    print("Using the records from the feature group")
except NameError:
    # If there is no feature group, use the processed dataset
    df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
    df_processed_pre_split = df_processed
    print("Using the processed records from Amazon S3")

# Split the data into train, validation, and test datasets
train_data, validation_data, test_data = np.split(
    df_processed_pre_split.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_processed_pre_split)), int(0.9 * len(df_processed_pre_split))],
)

# Create the CSV files and upload them to your default bucket
train_data.to_csv("train_data.csv", index=False, header=False)
validation_data.to_csv("validation_data.csv", index=False, header=False)
test_data.to_csv("test_data.csv", index=False, header=False)

train_path = S3Uploader.upload("train_data.csv", "s3://{}/{}".format(bucket, prefix))
validation_path = S3Uploader.upload("validation_data.csv", "s3://{}/{}".format(bucket, prefix))
test_path = S3Uploader.upload("test_data.csv", "s3://{}/{}".format(bucket, prefix))

# Set the training inputs
train_input = TrainingInput(train_path, content_type="text/csv")
validation_input = TrainingInput(validation_path, content_type="text/csv")
test_input = TrainingInput(test_path, content_type="text/csv")

data_inputs = {
    "train": train_input,
    "validation": validation_input
}

要继续本实验，请返回至<a href="#task3-2-continue" target="_self">任务 3.2</a>。

<a name="task3-3-1" id="task3-3-1"></a>

### 附录：配置并运行基本训练任务（任务 3.3）

如果您想从基本训练任务开始，请使用基本 **XGBoost** 容器。然后，配置估算器，记下您想要使用的容器和角色。设置了这些配置后，即可选择超参数。您可以使用下面的代码中提供的默认值，也可以根据您在数据准备期间的发现对其进行编辑。

要运行训练任务，请调用 **fit()**，将输入设置为 **data_inputs** 变量，并为 **run_name** 和 **experiment_name** 设置配置。

In [ ]:

from sagemaker import image_uris
#train-model
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8

hyperparameters = {
    "eta":eta,
    "gamma":gamma,
    "max_depth":max_depth,
    "min_child_weight":min_child_weight,
    "num_round":num_round,
    "objective":objective,
    "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role,    
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags
)

with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    )        

要继续本实验，请返回至<a href="#task3-3-continue" target="_self">任务 3.3</a>。

<a name="task3-3-2" id="task3-3-2"></a>

### 附录：在启用了 SageMaker Debugger 的情况下配置并运行训练任务以及分析报告（任务 3.3）

SageMaker Debugger 可以帮助您查看能够快速告知超参数优化情况的其他报告，从而在您开始运行更多具有超参数范围的训练任务时节省时间。要启用 Debugger，请配置 **DebuggerHookConfig** 和 **rules**。

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=300
objective='binary:logistic'
subsample=0.7
        
hyperparameters = {
        "eta":eta,
        "gamma":gamma,
        "max_depth":max_depth,
        "min_child_weight":min_child_weight,
        "num_round":num_round,
        "objective":objective,
        "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags,

    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
        ),
        #Set the Debugger Profiler Configuration
        profiler_config = ProfilerConfig(
            system_monitor_interval_millis=500,
            framework_profile_params=FrameworkProfile()
    ),
        #Configure the Debugger Rule Object
        rules = [
            ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
            Rule.sagemaker(rule_configs.create_xgboost_report()),  
            Rule.sagemaker(rule_configs.overfit()),
            Rule.sagemaker(rule_configs.overtraining()),
            Rule.sagemaker(rule_configs.loss_not_decreasing(),
                rule_parameters={
                    "collection_names": "metrics",
                    "num_steps": str(save_interval * 2),
                }
            )
    ]
)
with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)
# Train the model
xgb.fit(
    inputs = data_inputs
) 

要继续本实验，请返回至<a href="#task3-3-continue" target="_self">任务 3.3</a>。

<a name="task3-4" id="task3-4"></a>

### 附录：配置训练超参数范围（任务 3.4）

现在您已经训练了至少一个模型，接下来可以使用从数据处理和 SageMaker Debugger 中了解到的信息来确定您为超参数选择的范围。编辑以下超参数范围并运行优化任务。

In [ ]:
#tune-model
# Setup the hyperparameter ranges
hyperparameter_ranges = {
    "alpha": ContinuousParameter(0, 2),
    "eta": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(1, 10),
    "min_child_weight": ContinuousParameter(1, 10),
    "num_round": IntegerParameter(100, 1000)
}

# Define the target metric and the objective type (max/min)
objective_metric_name = "validation:auc"
objective_type="Maximize"

# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type="Auto"
)

# Tune the model
tuner.fit(
    inputs = data_inputs
)

要继续本实验，请返回至<a href="#task3-4-continue" target="_self">任务 3.4</a>。

<a name="task4-1" id="task4-1"></a>

### 附录：创建模型（任务 4.1）

创建 XGBoost 模型，使用您定义的 **model_name**、**role** 和 **container_def** 调用 **create_model**。

In [ ]:
#model-configurations
model_name = "capstone-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)

要继续本实验，请返回至<a href="#task4-1-continue" target="_self">任务 4.1</a>。

<a name="task4-2" id="task4-2"></a>

### 附录：创建 SageMaker Clarify 模型配置（任务 4.2）

使用 **SageMakerClarifyProcessor** 创建 SageMaker Clarify 模型配置。设置 **instance_count** 和 **instance_type**。使用在结业设计实验开始时创建的 **role** 和 **session**。

In [ ]:
#define-clarify-processor
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=sagemaker_session
)

要继续本实验，请返回至<a href="#task4-2-continue" target="_self">任务 4.2</a>。

<a name="task4-3" id="task4-3"></a>

### 附录：创建 SageMaker Clarify 偏差配置（任务 4.3）

要创建 SageMaker Clarify 偏差配置，请选择数据的输出路径，设置训练任务的输入路径以及 **label**、**headers** 和 **dataset_type**。

然后，创建 **ModelConfig** 和 **ModelPredictedLabelConfig**。

最后，配置 **BiasConfig**，其中包含您希望 SageMaker Clarify 观察的字段。您可以根据最初的发现添加或删除您想要探索的任何字段。

In [ ]:
#define-data-config
bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="fraud",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)


In [ ]:
#define-model-config
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [ ]:
#define-label-config
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

In [ ]:
#define-bias-config
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="customer_gender_female", facet_values_or_threshold=[0], group_name="customer_age"
)

要继续本实验，请返回至<a href="#task4-3-continue" target="_self">任务 4.3</a>。

<a name="task4-4" id="task4-4"></a>

### 附录：使用 SageMaker Clarify 运行偏差、数据和模型报告（任务 4.4）

现在您已经配置了 SageMaker Clarify 任务，接下来可以通过调用 **run_bias** 来运行任务。

In [ ]:
#run-bias-report
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

要继续本实验，请返回至<a href="#task4-4-continue" target="_self">任务 4.4</a>。

<a name="task4-5" id="task4-5"></a>

### 附录：消除不平衡（任务 4.5）

在本示例中，您要对 **customer_gender_female** 进行上采样，以减少数据集中的偏差。如果您发现其他包含偏差的特征，也可以消除这些特征中的不平衡。**random_state** 已设置为 `42`，但您可以更改此值。

In [ ]:
#display-summary
gender = train_data["customer_gender_female"]
gender.value_counts()

In [ ]:
#remove-imbalance
sm = SMOTE(random_state=42)
train_data_upsampled, gender_res = sm.fit_resample(train_data, gender)
train_data_upsampled["customer_gender_female"].value_counts()

要继续本实验，请返回至<a href="#task4-5-continue" target="_self">任务 4.5</a>。

<a name="task4-6" id="task4-6"></a>

### 附录：重新训练模型（任务 4.6）

您发现了不平衡，且有一个新的训练数据集。使用此数据集并重新训练文件。为此，请上传新文件并创建新的估算器。然后，使用 **fit()** 重新训练数据。下面的代码中包含了几个超参数作为样本。在重新训练期间，您可以根据需要添加、删除或调整这些超参数，从而找到最佳模型。

In [ ]:
#upload-upsampled-csv
train_data_upsampled.to_csv("data/upsampled_train.csv", index=False, header=False)
retrain_path = S3Uploader.upload("data/upsampled_train.csv", "s3://{}/{}".format(bucket, prefix))
retrain_input = TrainingInput(retrain_path, content_type="text/csv")

retrain_data_inputs = {
    "train": retrain_input,
    "validation": validation_input
}

In [ ]:
#create-estimator
hyperparameters= {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "300",
}

xgb_retrained = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters
)

In [ ]:
#retrain-upsampled-data
xgb_retrained.fit(
    inputs = retrain_data_inputs
) 

要继续本实验，请返回至<a href="#task4-6-continue" target="_self">任务 4.6</a>。

<a name="task5-1" id="task5-1"></a>

### 附录：创建批量转换任务（任务 5.1）

使用模型估算器通过**转换器**创建批量转换任务。将策略设置为 **MultiRecord** 以提高处理效率。然后，传入 **test_path** 并等待推理运行。

In [ ]:
#create-batch-transformer
# Use the retrained model if it exists, otherwise, use the original model
try:
    model = xgb_retrained
except NameError:
    model = xgb

# Create the transformer
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    strategy="MultiRecord",
    assemble_with="Line",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/".format(bucket, prefix)
)

In [ ]:
#run-batch-transform-job
test_data_batch = test_data.drop("fraud", axis=1)
test_data_batch.to_csv("test_data_batch.csv", index=False, header=False)
test_path_batch = S3Uploader.upload("test_data_batch.csv", "s3://{}/{}".format(bucket, prefix))

transformer.transform(test_path_batch, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

要继续本实验，请返回至<a href="#task5-1-continue" target="_self">任务 5.1</a>。

<a name="task5-2" id="task5-2"></a>

### 附录：查看来自批量转换任务的预测和准确率数据（任务 5.2）

完成批量转换任务后，查看存储在 Amazon S3 中的预测数据。您可以引用在**转换器**中设置的输出路径，并对数据进行采样。

在此输出中，每条记录的末尾都附加了 **fraud** 预测。

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
test_data = pd.read_csv("test_data_batch.csv.out")
test_data.head()

要继续本实验，请返回至<a href="#task5-2-continue" target="_self">任务 5.2</a>。

<a name="task5-3" id="task5-3"></a>

### 附录：在 SageMaker Studio 中清理 SageMaker 实例（任务 5.3）

在 SageMaker Studio 中开发模型时，请进行定期检查以了解是否有任何需要清理的实例。如果有，您可以在 SageMaker Studio 中关闭实例。

1.在左侧菜单栏中，选择**正在运行的终端和内核**图标（中间有一个正方形的圆圈）。

1.如果有任何实例仍然处于打开状态，在每个实例类型的右侧，选择**关闭**图标。

您可以查看每个实例上正在运行的应用程序，以确认要关闭哪些应用程序。

1.如果出现弹出窗口，请选择 **Shut down all**（关闭全部）。

1.时不时选择**刷新列表**图标，直到列表中已不再有相应实例。关闭实例可能需要 2–5 分钟。

您不需要关闭 **capstone.ipynb** 笔记本正在使用的实例。

要继续本实验，请返回至<a href="#task5-3-continue" target="_self">任务 5.3</a>。

<a name="task6-1" id="task6-1"></a>

### 附录：配置管道（任务 6.1）

要创建管道，请定义管道流程的每个步骤，然后运行该管道。

在本示例中，您将创建以下步骤：
- **AutoModelProcess**：在 .csv 文件中拉取数据并将其分割成训练、测试和验证数据集的**处理**步骤。
- **AutoHyperParameterTuning**：使用一系列超参数并优化模型的**优化**步骤。
- **AutoEvalBestModel**：创建评估报告来描述最佳模型的**处理**步骤。
- **CheckAUCScoreAutoEvaluation**：根据评估指标评估模型的**条件**步骤。
- **AutoCreateModel**：创建模型的**模型**步骤。
- **RegisterAutoModel-RegisterModel**：注册模型的**注册模型**步骤。
- **AutoModelConfigFile**：创建偏差报告的**处理**步骤。
- **AutoTransform**：运行批量转换任务的**转换**步骤。
- **ClarifyProcessingStep**：运行 SageMaker Clarify 任务的**处理**步骤。

如果您在构建管道时遇到问题，可以自定义以下代码，或者在构建自己的管道时将其作为指导。

In [ ]:
#run-pipeline
# Set the variables
model_name = "Auto-model"
sklearn_processor_version="0.23-1"
model_package_group_name="AutoModelPackageGroup"
pipeline_name= "AutoModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

# Upload files to the default S3 bucket
s3_client.put_object(Bucket=bucket,Key='data/')
s3_client.put_object(Bucket=bucket,Key='input/code/')
s3_client.upload_file(Filename="data/batch_data.csv", Bucket=bucket, Key="data/batch_data.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key="data/claims_customer.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="pipelines/evaluate.py", Bucket=bucket, Key="input/code/evaluate.py")
s3_client.upload_file(Filename="pipelines/generate_config.py", Bucket=bucket, Key="input/code/generate_config.py")
s3_client.upload_file(Filename="pipelines/preprocess.py", Bucket=bucket, Key="input/code/preprocess.py")

# Configure important settings. Change the input_data if you want to
# use a file other than the claims_customer.csv and batch_data.csv files.
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
        name="TrainingInstanceType",
        default_value="ml.m5.xlarge"
)
input_data = ParameterString(
        name="InputData",
        default_value="s3://{}/data/claims_customer.csv".format(bucket), 
)
batch_data = ParameterString(
        name="BatchData",
        default_value="s3://{}/data/batch_data.csv".format(bucket),
)

# Run a scikit-learn script to do data processing on SageMaker using 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version=sklearn_processor_version,
        instance_type=processing_instance_type.default_value, 
        instance_count=processing_instance_count,
        sagemaker_session=sagemaker_session,
        role=role,
)

# Configure the processing step to pull in the input_data
step_process = ProcessingStep(
        name="AutoModelProcess",
        processor=sklearn_processor,
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train",\
                             destination=f"s3://{bucket}/output/train" ),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation",\
                            destination=f"s3://{bucket}/output/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test",\
                            destination=f"s3://{bucket}/output/test"),
            ProcessingOutput(output_name="batch", source="/opt/ml/processing/batch",\
                            destination=f"s3://{bucket}/data/batch"),
            ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline",\
                            destination=f"s3://{bucket}/input/baseline")
        ],
        code=f"s3://{bucket}/input/code/preprocess.py",
        job_arguments=["--input-data", input_data],
)

# Set up the model path, image uri, and hyperparameters for the estimator
model_path = f"s3://{bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.5-1",
    py_version="py3",
    instance_type=training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
}

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    output_path=model_path,
    base_job_name=f"auto-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set the hyperparameter ranges for the tuning step and configure the tuning step
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "AutoHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs=2, max_parallel_jobs=2),
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
)

# Configure the processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-auto-eval",
    role=role,
    sagemaker_session=sagemaker_session,
)

evaluation_report = PropertyFile(
    name="AutoEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_eval = ProcessingStep(
    name="AutoEvalBestModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation",\
                            destination=f"s3://{bucket}/output/evaluation"),
    ],
    code=f"s3://{bucket}/input/code/evaluate.py",
    property_files=[evaluation_report],
)

# Configure model creation
model = Model(
    image_uri=image_uri,        
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    name=model_name,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name="AutoCreateModel",
    model=model,
    inputs=inputs,
)

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri=clarify_image,
    role=role,
    instance_count=1,
    instance_type=processing_instance_type,
    sagemaker_session=sagemaker_session,
)

bias_report_output_path = f"s3://{bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
step_config_file = ProcessingStep(
    name="AutoModelConfigFile",
    processor=script_processor,
    code=f"s3://{bucket}/input/code/generate_config.py",
    job_arguments=["--modelname",step_create_model.properties.ModelName,"--bias-report-output-path",bias_report_output_path,"--clarify-instance-type",clarify_instance_type,\
                  "--default-bucket",bucket,"--num-baseline-samples","50","--instance-count","1"],
    depends_on= [step_create_model.name]
)

# Configure the step to perform a batch transform job
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    assemble_with="Line",
    accept="text/csv",    
    output_path=f"s3://{bucket}/AutoTransform"
)

step_transform = TransformStep(
    name="AutoTransform",
    transformer=transformer,
    inputs=TransformInput(data=batch_data,content_type="text/csv",join_source="Input",split_type="Line")
)

# Configure the SageMaker Clarify processing step
analysis_config_path = f"s3://{bucket}/clarify-output/bias/analysis_config.json"

data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=f's3://{bucket}/output/train/train.csv', 
    s3_output_path=bias_report_output_path,
    label=0,
    headers=list(pd.read_csv("./data/claims_customer.csv", index_col=None).columns), #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
    dataset_type="text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type=clarify_instance_type,
    sagemaker_session=sagemaker_session,
)

config_input = ProcessingInput(
    input_name="analysis_config",
    source=analysis_config_path,
    destination="/opt/ml/processing/input/analysis_config",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_compression_type="None",
)

data_input = ProcessingInput(
    input_name="dataset",
    source=data_config.s3_data_input_path,
    destination="/opt/ml/processing/input/data",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type=data_config.s3_data_distribution_type,
    s3_compression_type=data_config.s3_compression_type,
)

result_output = ProcessingOutput(
    source="/opt/ml/processing/output",
    destination=data_config.s3_output_path,
    output_name="analysis_result",
    s3_upload_mode="EndOfJob",
)

step_clarify = ProcessingStep(
    name="ClarifyProcessingStep",
    processor=clarify_processor,
    inputs= [data_input, config_input],
    outputs=[result_output],
    depends_on = [step_config_file.name]
)

# Configure the model registration step
model_statistics = MetricsSource(
    s3_uri="s3://{}/output/evaluation/evaluation.json".format(bucket),
    content_type="application/json"
)
explainability = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
)

bias = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
) 

model_metrics = ModelMetrics(
    model_statistics=model_statistics,
    explainability=explainability,
    bias=bias
)

step_register = RegisterModel(
    name="RegisterAutoModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
)

# Create the model evaluation step
cond_lte = ConditionGreaterThan(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value"
    ),
    right=0.75,
)

step_cond = ConditionStep(
    name="CheckAUCScoreAutoEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model,step_config_file,step_transform,step_clarify,step_register],
    else_steps=[],
)

# Define the pipeline
def get_pipeline(
    region,
    role=None,
    default_bucket=None,
    model_package_group_name="AutoModelPackageGroup",
    pipeline_name="AutoModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version=None
    ):
    """Gets a SageMaker ML Pipeline instance working with auto data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """

    # pipeline instance
    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps=[step_process,step_tuning,step_eval,step_cond],
        sagemaker_session=sagemaker_session
    )
    return pipeline


# Create the pipeline
pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

pipeline.upsert(role_arn=role)

# Run the pipeline
RunPipeline = pipeline.start()

要继续本实验，请返回至<a href="#task6-1-continue" target="_self">任务 6.1</a>。

<a name="task6-2" id="task6-2"></a>

### 附录：监控管道（任务 6.2）

现在您已经创建并运行了管道，接下来可以监控管道。您可以在 SageMaker Studio 中查看管道状态。

如果您要删除管道，可以使用 **delete_pipeline** 将其删除。

In [ ]:
#describe-pipeline
RunPipeline.describe()

In [ ]:
#list-pipeline-steps
RunPipeline.list_steps()

In [ ]:
#remove-pipeline
response = sagemaker_session.boto_session.client("sagemaker", region_name=region).delete_pipeline(PipelineName='AutoModelSMPipeline')
print(response)

要继续本实验，请返回至<a href="#task6-2-continue" target="_self">任务 6.2</a>。